<a href="https://colab.research.google.com/github/ProtossDragoon/CameraCalibration/blob/master/CoreML_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 8.3 MB/s 


In [2]:
import tensorflow as tf

# tf.keras를 이용해 MobileNet v2를 다운로드한다.
keras_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    input_shape=(224, 224, 3),
    classes=1000,
)

# 클래스 레이블을 다운로드한다.
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().splitlines() 
class_labels = class_labels[1:] # 0번째 인덱스는 'background' 클래스이므로 포함하지 않는다.
assert len(class_labels) == 1000

# 클래스 레이블이 모두 문자열이 되도록 한 번 더 확인한다.
for i, label in enumerate(class_labels):
    if isinstance(label, bytes):
        class_labels[i] = label.decode("utf8")

14536120/14536120 [==============================] - 0s 0us/step


In [3]:
import coremltools as ct

# 입력 데이터 타입을 이미지로 정의한다.
image_input = ct.ImageType(
    shape=(1, 224, 224, 3),
    bias=[-1, -1, -1], # 픽셀값을 [-1, 1]의 범위로 변환하는 정규화 전처리를 사용한다. mobilenet 모델이 학습될 당시 사용했던 전처리 특성과 동일한 상태로 만들기 위함이다.
    scale=1/127
)

# 클래스 레이블을 설정한다.
classifier_config = ct.ClassifierConfig(class_labels)

# CoreML 도구가 제공하는 변환 API 를 이용하여 모델을 변환한다.
model = ct.convert(
    keras_model,
    inputs=[image_input],
    classifier_config=classifier_config,
)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 487/487 [00:00<00:00, 1298.07 ops/s]


In [9]:
print(model.get_spec().description)

input {
  name: "input_1"
  shortDescription: "Input image to be classified"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "Identity"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  shortDescription: "Most likely image category"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "Identity"
metadata {
  versionString: "2.0"
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.9.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.1"
  }
}



In [38]:
# 아래 설정한 내용들은 XCode에서 모델에 대한 설명을 표시하는 데 사용된다.

# 입출력에 대한 설명을 붙여 준다.
model.input_description["input_1"] = "입력 이미지"
model.output_description["classLabel"] = "이미지 카테고리"

# 모델 저자 정보를 입력한다. 
model.author = "Practical-MLOps"

# 모델의 라이센스 정보를 입력한다.
model.license = "Apache2.0"

# 짧은 설명을 추가한다.
model.short_description = "Practical-MLOps Chapter5"

# 모델의 버전을 설정한다.
model.version = "2.0"

In [ ]:
print(model.get_spec().description)

In [39]:
# 코랩 컴퓨터에 모델을 저장한다.
model.save("MobileNetV2.mlmodel")